In [1]:
# Read in the document-term matrix
import pandas as pd

data = pd.read_pickle('dtm.pkl')
data = data.transpose()
data.head()

,obama,trump
abject,0,1
able,1,3
abolish,0,1
abortion,0,1
abroad,1,1


In [2]:
# Find the top 30 words said by each comedian
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{'obama': [('applause', 84),
  ('thats', 35),
  ('people', 27),
  ('america', 27),
  ('just', 25),
  ('world', 24),
  ('american', 22),
  ('work', 22),
  ('want', 19),
  ('new', 19),
  ('make', 19),
  ('change', 18),
  ('years', 17),
  ('year', 17),
  ('economy', 16),
  ('weve', 15),
  ('need', 15),
  ('better', 15),
  ('americans', 15),
  ('dont', 14),
  ('like', 13),
  ('right', 13),
  ('way', 13),
  ('future', 12),
  ('country', 12),
  ('job', 11),
  ('security', 10),
  ('going', 10),
  ('im', 10),
  ('know', 9)],
 'trump': [('american', 33),
  ('new', 23),
  ('years', 21),
  ('united', 21),
  ('states', 21),
  ('tonight', 19),
  ('country', 17),
  ('americans', 17),
  ('america', 15),
  ('congress', 14),
  ('border', 13),
  ('decades', 12),
  ('women', 12),
  ('come', 11),
  ('just', 11),
  ('time', 11),
  ('year', 11),
  ('nation', 10),
  ('illegal', 10),
  ('jobs', 10),
  ('americas', 10),
  ('work', 10),
  ('administration', 9),
  ('people', 9),
  ('world', 9),
  ('make', 8),
  